<a href="https://colab.research.google.com/github/micheldc55/cheatsheet/blob/main/03_clasificador_caras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
COLAB = True

In [ ]:
!ls -la

In [ ]:
if COLAB:
    from google_drive_downloader import GoogleDriveDownloader as gdd
    gdd.download_file_from_google_drive(file_id='1q2shS6fBq_pSI8CBrjzdgg6DTTZfZ2mJ',
                                        dest_path='./funciones_auxiliares.py')
    gdd.download_file_from_google_drive(file_id='1qV3hqJFajv6iYLVqwtMIvr6Mo_GOOdQv',
                                        dest_path='./caras_aux.py')
    gdd.download_file_from_google_drive(file_id='1d_94YSDZu662q3JJ17r1dC1BHfOCTKZU',
                                        dest_path='./datasets/gender.tgz')
    !tar -xvf datasets/gender.tgz
    !mv gender datasets

In [ ]:
!ls

In [ ]:
!ls datasets

In [ ]:
!ls -la datasets/gender

In [ ]:
!ls -la datasets/gender/female

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import roc_curve, auc, confusion_matrix

%matplotlib inline

La base de datos usada es un subconjunto de la base de datos
"Labeled Faces in the Wild" ("LFW"):

  http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz

  http://vis-www.cs.umass.edu/lfw/

La separación en carpetas "female" / "male" se ha realizado usando
un código basado en:
https://github.com/Pletron/LFWgender

In [ ]:
from caras_aux import loadImages, plot_gallery

# cargo las imágenes y preparo los datos que se van a usar
factor = 0.15 # factor de reducción de resolución de las imágenes, debe ser
             # >0 y <=1 (en cuyo caso la resolución es la original, 250x250)
X_clase0, h, w = loadImages('./datasets/gender/female',
                            factor=factor, nimages=1000)
X_clase1, h, w = loadImages('./datasets/gender/male',
                            factor=factor, nimages=1000)
X = np.concatenate((X_clase0, X_clase1))
n_samples, n_features = X.shape

target_names = np.array(['female', 'male'])
n_classes = len(target_names)
N0 = len(X_clase0); N1 = len(X_clase1)
y = N0*[0] + N1*[1]
y = np.array(y)

print('\nEstadísticas básicas de la base de datos leída:')
print('  # clases: %d' % n_classes)
print('  # patrones: %d' % n_samples)
print('  # atributos (features): %d' % n_features)

print('\nAlgunas imágenes de la clase "female":')
plot_gallery(X_clase0[:12].reshape((12, h, w)), 12*['female'], cmap=plt.cm.gray)
print('\nAlgunas imágenes de la clase "male":')
plot_gallery(X_clase1[:12].reshape((12, h, w)), 12*['male'], cmap=plt.cm.gray)

In [ ]:
print(np.shape(X)); print(np.shape(y)); print(X)

In [ ]:
np.sqrt(1369)

In [ ]:
X.min(), X.max()

In [ ]:
X /= 255

In [ ]:
np.unique(y)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
    
# divido en conjuntos de training y test
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=200,
                                                    random_state=42)

In [ ]:
from caras_aux import report_base_error

plt.figure(figsize=(10,3))
ax = plt.subplot(1,2,1)
clases, counts = np.unique(y_train, return_counts=True)
plt.bar(clases[0], 100*counts[0]/len(y_train))
plt.bar(clases[1], 100*counts[1]/len(y_train))
plt.title('Training'); plt.xlabel('Clase'); plt.ylabel('Frequency (%)'); ax.set_xticks(clases)

ax = plt.subplot(1,2,2)
none, counts = np.unique(y_test, return_counts=True)
plt.bar(clases[0], 100*counts[0]/len(y_test))
plt.bar(clases[1], 100*counts[1]/len(y_test))
plt.title('Test'); plt.xlabel('Clase'); ax.set_xticks(clases); plt.show()

report_base_error(X_train, y_train, X_test, y_test)

Otra forma de hacer el mismo cálculo:

In [ ]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier(strategy='prior')
clf.fit(X=X_train, y=y_train)
print('Score del clasificador Dummy en test:', clf.score(X=X_test,y=y_test)) # en realidad no usa X_test

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model

In [ ]:
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred  = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_test,  y_test_pred))

In [ ]:
from funciones_auxiliares import analisis_roc, plot_confusion_matrix

In [ ]:
# Resultados detallados en test
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
analisis_roc(y_test, model.predict_proba(X_test)[:,1])
plt.subplot(1,2,2)
plot_confusion_matrix(y_test, y_test_pred)